In [62]:
import sys
import datetime
import hashlib
import json
from flask import Flask, jsonify , request

import requests
from uuid import uuid4
from urllib.parse import urlparse


In [63]:
from urllib.parse import urlparse
address = 'http://127.0.0.1:5001'
parse_url = urlparse(address)
parse_url

ParseResult(scheme='http', netloc='127.0.0.1:5001', path='', params='', query='', fragment='')

In [64]:
class Blockchain:
    def __init__(self):
        self.chain = []
        self.transactions = []
        self.create_block(proof=1, previous_hash='0')
        self.nodes = set()

    def add_node(self, address):
        parse_url = urlparse(address)
        self.nodes.add(parse_url.netloc)
    
    def replace_chain(self):
        network = self.nodes
        longest_chain = None
        max_length = len(self.chain)

        for node in network:
            print(f'http://{node}/get_chain')
            response = requests.get(f'http://{node}/get_chain')
            if response.status_code==200:
                length = response.json()['length']
                print(length)
                chain = response.json()['chain']
                if length > max_length and self.is_chain_valid(chain):
                    longest_chain = chain
                    max_length = length
            
        if longest_chain:
            self.chain = longest_chain
            return True
        return False 

    def create_block(self, proof, previous_hash):
        block = {'index':len(self.chain)+1,
                 'timestamp':str(datetime.datetime.now()),
                 'proof':proof,
                 'previous_hash':previous_hash,
                 'transactions':self.transactions}
        
        self.transactions = []
        self.chain.append(block)
        return block
    
    def add_transaction(self, sender, recevier, amount):
        self.transactions.append({'sender':sender , 'recevier':recevier, 'amount':amount})
        previous_block = self.get_previous_block()
        return previous_block["index"]-1

    def get_previous_block(self):
        return self.chain[-1]
    
    def proof_of_work(self, previous_proof):
        new_proof = 1
        check_proof = False

        try:
            while check_proof is False:
                hash_operation = hashlib.sha256(str(new_proof**2 - previous_proof**2).encode()).hexdigest()
                if hash_operation[:4] == '0000':
                    check_proof = True
                else:
                    new_proof += 1
                
            return  new_proof
        
        except Exception as e:        
            print(f'\t - ::Error:: error mining : {str(e)}') 
            #sys.exit(1)
        
    def hash(self, block):
        encoded_block = json.dumps(block, sort_keys = True).encode()
        return hashlib.sha256(encoded_block).hexdigest()
    
    def is_chain_valid(self, chain):
        previous_block = chain[0]
        block_index = 1
        while block_index < len(chain):
            block = chain[block_index]
            if block['previous_hash'] != self.hash(previous_block):
                return False
            
            previous_proof = previous_block['proof']
            proof = block['proof']
            hash_operation = hashlib.sha256(str(proof**2 - previous_proof**2).encode()).hexdigest()
            if hash_operation[:4] != '0000':
                return False

            previous_block = block
            block_index += 1

        return True
    

    

In [65]:
app = Flask(__name__)
blockchain = Blockchain()

# Creando una dirección para el nodo en puerto 5001
node_address = str(uuid4()).replace('-','')

# Minando bloque
@app.route('/mine_block', methods=['GET'])

def mine_block():
    previous_block = blockchain.get_previous_block()    
    previous_proof = previous_block['proof']

    proof = blockchain.proof_of_work(previous_proof)
    print(f'::info:: Mine OK')
    blockchain.add_transaction(sender=node_address, recevier='Moostay', amount=1)
    previous_hash = blockchain.hash(previous_block)
    block = blockchain.create_block(proof, previous_hash)
    response = {"message":'You has mined a block',
                'index': block['index'],
                'timestamp':block['timestamp'],
                'proof':block['proof'],
                'previous_hash':block['previous_hash'],
                'transactions':block['transactions']}
    return jsonify(response),200

@app.route('/get_chain', methods=['GET'])
def get_chain():
    response = {'chain': blockchain.chain,
                'length':len(blockchain.chain)}
    return jsonify(response), 200

@app.route('/is_valid', methods=['GET'])
def is_valid():
    is_valid = blockchain.is_chain_valid(blockchain.chain)
    if is_valid:
        response = {'message':'Bloackchain valid'}
    else:
        response = {'message':'Bloackchain NO valid'}
    return jsonify(response),200


In [66]:
# Agregar transacción al blockchain
@app.route('/add_transaction', methods=['POST'])
def add_transaction():
    json = request.get_json()
    transaction_keys = ['sender','recevier','amount']
    if not all (key in json for key in transaction_keys):
        return'Algún elemento de la transacción está faltando',400
    
    index = blockchain.add_transaction(json['sender'], json['recevier'], json['amount'])
    response = {'message':f'La transacción será añadida al bloque: {index}'}
    return jsonify(response), 201    #-201 porque es port request

# Descentralizando el blockchain

#Conectando nuevos nodos
@app.route('/connect_node', methods=['POST'])
def connect_node():
    json = request.get_json()
    nodes = json.get('nodes')
    if nodes is None:
        return "No node", 401
    for node in nodes:
        blockchain.add_node(node)
    response = {'message':'Todos los nodos estan ahora conectados',
                'total_nodes':list(blockchain.nodes)}
    return jsonify(response), 201
    

# remplazando la cadena por la mas larga
@app.route('/replace_chain', methods=['GET'])
def replace_chain():
    is_chain_replace = blockchain.replace_chain()
    if is_chain_replace:
        response = {'message':'Nodes replaced valid', 'new_chain':blockchain.chain}
    else:
        response = {'message':'The chain is OK' , 'actual_chain':blockchain.chain}
    return jsonify(response),200

In [67]:
app.run(host='0.0.0.0', port=5001)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5001
 * Running on http://192.168.20.61:5001
Press CTRL+C to quit
127.0.0.1 - - [21/Dec/2023 23:12:27] "GET /get_chain HTTP/1.1" 200 -
127.0.0.1 - - [21/Dec/2023 23:12:33] "POST /connect_node HTTP/1.1" 201 -
127.0.0.1 - - [21/Dec/2023 23:12:42] "GET /replace_chain HTTP/1.1" 200 -


http://127.0.0.1:5002/get_chain
1
http://127.0.0.1:5000/get_chain
2


127.0.0.1 - - [21/Dec/2023 23:12:57] "GET /get_chain HTTP/1.1" 200 -
127.0.0.1 - - [21/Dec/2023 23:15:32] "GET /get_chain HTTP/1.1" 200 -
127.0.0.1 - - [21/Dec/2023 23:15:52] "GET /mine_block HTTP/1.1" 200 -


::info:: Mine OK


127.0.0.1 - - [21/Dec/2023 23:16:14] "GET /get_chain HTTP/1.1" 200 -
127.0.0.1 - - [21/Dec/2023 23:17:40] "GET /get_chain HTTP/1.1" 200 -
127.0.0.1 - - [21/Dec/2023 23:18:07] "GET /get_chain HTTP/1.1" 200 -
127.0.0.1 - - [21/Dec/2023 23:18:28] "GET /replace_chain HTTP/1.1" 200 -


http://127.0.0.1:5002/get_chain
3
http://127.0.0.1:5000/get_chain
3


127.0.0.1 - - [21/Dec/2023 23:19:19] "GET /get_chain HTTP/1.1" 200 -
127.0.0.1 - - [21/Dec/2023 23:20:10] "GET /get_chain HTTP/1.1" 200 -
127.0.0.1 - - [21/Dec/2023 23:20:42] "GET /get_chain HTTP/1.1" 200 -
127.0.0.1 - - [21/Dec/2023 23:20:43] "GET /get_chain HTTP/1.1" 200 -
127.0.0.1 - - [21/Dec/2023 23:20:47] "GET /get_chain HTTP/1.1" 200 -
127.0.0.1 - - [21/Dec/2023 23:20:48] "GET /get_chain HTTP/1.1" 200 -
